# RAPIDS & Scanpy Single-Cell RNA-seq Workflow

Copyright (c) 2020, NVIDIA CORPORATION.

Licensed under the Apache License, Version 2.0 (the "License") you may not use this file except in compliance with the License. You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0 

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.

This notebook demonstrates a single-cell RNA analysis workflow that begins with preprocessing a count matrix of size `(n_gene, n_cell)` and results in a visualization of the clustered cells for further analysis.

For demonstration purposes, we use a dataset of ~70,000 human lung cells from Travaglini et al. 2020 (https://www.biorxiv.org/content/10.1101/742320v2) and label cells using the ACE2 and TMPRSS2 genes. See the README for instructions to download this dataset.

## Import requirements

In [1]:
import numpy as np
import scanpy as sc
import anndata

import sys
import time
import os

import cudf
import cupy as cp

from cuml.decomposition import PCA
from cuml.manifold import TSNE
from cuml.cluster import KMeans

import rapids_scanpy_funcs

import warnings
warnings.filterwarnings('ignore', 'Expected ')

import cuml
import sklearn

In [2]:
import rmm

rmm.reinitialize(
    managed_memory=True, # Allows oversubscription
    pool_allocator=False, # default is False
    devices=0, # GPU device IDs to register. By default registers only GPU 0.
)

cp.cuda.set_allocator(rmm.rmm_cupy_allocator)

## Input data

In [3]:
input_file = "../data/krasnow_hlca_10x_UMIs.sparse.h5ad"

## Set parameters

In [4]:
# marker genes
RIBO_GENE_PREFIX = "RPS" # Prefix for ribosomal genes to regress out
markers = ["ACE2", "TMPRSS2", "EPCAM"] # Marker genes for visualization

# filtering cells
min_genes_per_cell = 200 # Filter out cells with fewer genes than this expressed 
max_genes_per_cell = 6000 # Filter out cells with more genes than this expressed 

# filtering genes
n_top_genes = 5000 # Number of highly variable genes to retain

# PCA
n_components = 50 # Number of principal components to compute

# t-SNE
tsne_n_pcs = 20 # Number of principal components to use for t-SNE

# k-means
k = 20 # Number of clusters for k-means

# KNN
n_neighbors = 15 # Number of nearest neighbors for KNN graph
knn_n_pcs = 50 # Number of principal components to use for finding nearest neighbors

# UMAP
umap_min_dist = 0.3 
umap_spread = 1.0

# Louvain
louvain_resolution = 0.4

# Gene ranking
ranking_n_top_genes = 50 # Number of differential genes to compute for each cluster

In [5]:
start = time.time()

## Load and Prepare Data

We load the sparse count matrix from an `h5ad` file using Scanpy. The sparse count matrix will then be placed on the GPU. 

In [6]:
data_load_start = time.time()

In [7]:
%%time
adata = sc.read(input_file)
adata = adata.T

CPU times: user 4.93 s, sys: 739 ms, total: 5.67 s
Wall time: 5.67 s


In [8]:
adata.shape

(65662, 26485)

We maintain the index of unique cells and genes in our dataset:

In [9]:
%%time
cells = cudf.Series(adata.obs_names)
genes = cudf.Series(adata.var_names)

CPU times: user 685 ms, sys: 490 ms, total: 1.17 s
Wall time: 1.29 s


In [10]:
%%time
sparse_gpu_array = cp.sparse.csr_matrix(adata.X)

CPU times: user 171 ms, sys: 547 ms, total: 718 ms
Wall time: 723 ms


Verify the shape of the resulting sparse matrix:

In [11]:
sparse_gpu_array.shape

(65662, 26485)

And the number of non-zero values in the matrix:

In [12]:
sparse_gpu_array.nnz

126510394

In [13]:
data_load_time = time.time()
print("Total data load and format time: %s" % (data_load_time-data_load_start))

Total data load and format time: 7.722462177276611


## Preprocessing

In [14]:
preprocess_start = time.time()

### Filter

We filter the count matrix to remove cells with an extreme number of genes expressed.

In [15]:
%%time
sparse_gpu_array = rapids_scanpy_funcs.filter_cells(sparse_gpu_array, min_genes=min_genes_per_cell, max_genes=max_genes_per_cell)

CPU times: user 637 ms, sys: 745 ms, total: 1.38 s
Wall time: 1.43 s


Some genes will now have zero expression in all cells. We filter out such genes.

In [16]:
%%time
sparse_gpu_array, genes = rapids_scanpy_funcs.filter_genes(sparse_gpu_array, genes, min_cells=1)

CPU times: user 1.05 s, sys: 227 ms, total: 1.28 s
Wall time: 1.29 s


The size of our count matrix is now reduced.

In [17]:
sparse_gpu_array.shape

(65462, 22058)

### Normalize

We normalize the count matrix so that the total counts in each cell sum to 1e4.

In [18]:
%%time
sparse_gpu_array = rapids_scanpy_funcs.normalize_total(sparse_gpu_array, target_sum=1e4)

CPU times: user 928 µs, sys: 483 µs, total: 1.41 ms
Wall time: 873 µs


Next, we log transform the count matrix.

In [19]:
%%time
sparse_gpu_array = sparse_gpu_array.log1p()

CPU times: user 86.8 ms, sys: 63.6 ms, total: 150 ms
Wall time: 149 ms


### Select Most Variable Genes

We convert the count matrix to an annData object.

In [20]:
%%time
adata = anndata.AnnData(sparse_gpu_array.get())
adata.var_names = genes.to_pandas()

CPU times: user 278 ms, sys: 246 ms, total: 524 ms
Wall time: 522 ms


Using scanpy, we filter the count matrix to retain only the 5000 most variable genes.

In [21]:
%%time
sc.pp.highly_variable_genes(adata, n_top_genes=n_top_genes, flavor="cell_ranger")
adata = adata[:, adata.var.highly_variable]

CPU times: user 1.94 s, sys: 34.4 ms, total: 1.98 s
Wall time: 2.24 s


### Regress out confounding factors (number of counts, ribosomal gene expression)

We can now perform regression on the count matrix to correct for confounding factors -  for example purposes, we use the number of counts and the expression of ribosomal genes. Many workflows use the expression of mitochondrial genes (named starting with `MT-`).

Before regression, we save the 'raw' expression values of the ACE2 and TMPRSS2 genes to use for labeling cells afterward. We will also store the expression of an epithelial marker gene (EPCAM).

In [22]:
%%time
tmp_norm = sparse_gpu_array.tocsc()
ACE2_raw = tmp_norm[:, genes[genes == "ACE2"].index[0]].todense().ravel()
TMPRSS2_raw = tmp_norm[:, genes[genes == "TMPRSS2"].index[0]].todense().ravel()
EPCAM_raw = tmp_norm[:, genes[genes == "EPCAM"].index[0]].todense().ravel()

del tmp_norm

CPU times: user 521 ms, sys: 232 ms, total: 752 ms
Wall time: 779 ms


In [23]:
genes = adata.var_names
ribo_genes = adata.var_names.str.startswith(RIBO_GENE_PREFIX)

In [24]:
%%time
filtered = adata.X

CPU times: user 544 ms, sys: 26 ms, total: 570 ms
Wall time: 569 ms


We now calculate the total counts and the percentage of ribosomal counts for each cell.

In [25]:
%%time
n_counts = filtered.sum(axis=1)
percent_ribo = (filtered[:,ribo_genes].sum(axis=1) / n_counts).ravel()

n_counts = cp.array(n_counts).ravel()
percent_ribo = cp.array(percent_ribo).ravel()

CPU times: user 56.3 ms, sys: 0 ns, total: 56.3 ms
Wall time: 54.5 ms


And perform regression:

In [26]:
%%time
sparse_gpu_array = cp.sparse.csc_matrix(adata.X)
sparse_gpu_array = rapids_scanpy_funcs.regress_out(sparse_gpu_array, n_counts, percent_ribo)

CPU times: user 1min, sys: 25.2 s, total: 1min 25s
Wall time: 1min 50s


### Scale

Finally, we scale the count matrix to obtain a z-score and apply a cutoff value of 10 standard deviations, obtaining the preprocessed count matrix.

In [27]:
%%time
sparse_gpu_array = rapids_scanpy_funcs.scale(sparse_gpu_array, max_value=10)

CPU times: user 180 ms, sys: 174 ms, total: 354 ms
Wall time: 352 ms


## Cluster & Visualize

In [28]:
cluster_start = time.time()

We store the preprocessed count matrix as an AnnData object, which is currently in host memory. We also add the expression levels of the marker genes as observations to the annData object.

In [29]:
%%time

var_names = adata.var_names
adata = anndata.AnnData(sparse_gpu_array.get())
adata.var_names = var_names
adata.obs["ACE2_raw"] = ACE2_raw.get()
adata.obs["TMPRSS2_raw"] = TMPRSS2_raw.get()
adata.obs["EPCAM_raw"] = EPCAM_raw.get()

CPU times: user 329 ms, sys: 294 ms, total: 623 ms
Wall time: 622 ms


### Reduce

We use PCA to reduce the dimensionality of the matrix to its top 50 principal components.

In [30]:
%%time
adata.obsm["X_pca"] = PCA(n_components=n_components, output_type="numpy").fit_transform(adata.X)

CPU times: user 1.42 s, sys: 793 ms, total: 2.21 s
Wall time: 2.21 s


We visualize the cells using t-SNE and label cells by color according to the k-means clustering.

### UMAP + Louvain

We can also visualize the cells using the UMAP algorithm in Rapids. Before UMAP, we need to construct a k-nearest neighbors graph in which each cell is connected to its nearest neighbors. This can be done conveniently using rapids functionality already integrated into Scanpy.

Note that Scanpy uses an approximation to the nearest neighbors on the CPU while the GPU version performs an exact search. While both methods are known to yield useful results, some differences in the resulting visualization and clusters can be observed.

In [31]:
%%time
sc.pp.neighbors(adata, n_neighbors=n_neighbors, n_pcs=knn_n_pcs, method='rapids')

/rapids-genomics/rapidgenomics/lib/python3.7/site-packages/numba/np/ufunc/parallel.py:355: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 10005. The TBB threading layer is disabled.
  warnings.warn(problem)


CPU times: user 8.16 s, sys: 740 ms, total: 8.9 s
Wall time: 7 s


The UMAP function from Rapids is also integrated into Scanpy.

In [32]:
%%time
sc.tl.umap(adata, min_dist=umap_min_dist, spread=umap_spread, method='rapids')

CPU times: user 584 ms, sys: 525 ms, total: 1.11 s
Wall time: 1.14 s


Finally, we use the Louvain algorithm for graph-based clustering, once again using the `rapids` option in Scanpy.

In [33]:
%%time
sc.tl.louvain(adata, resolution=louvain_resolution, flavor='rapids')

/rapids-genomics/rapidgenomics/lib/python3.7/site-packages/cugraph/structure/graph.py:333: UserWarning: add_adj_list will be deprecated in next release. Use from_cudf_adjlist instead
  Use from_cudf_adjlist instead')


CPU times: user 267 ms, sys: 181 ms, total: 448 ms
Wall time: 770 ms


## Differential expression analysis

Once we have done clustering, we can compute a ranking for the highly differential genes in each cluster. Here we use the Louvain clusters as labels.

In [34]:
def select_groups(labels, groups_order_subset='all'):
    adata_obs_key = labels
    groups_order = labels.cat.categories
    groups_masks = cp.zeros(
        (len(labels.cat.categories), len(labels.cat.codes)), dtype=bool
    )
    for iname, name in enumerate(labels.cat.categories):
        # if the name is not found, fallback to index retrieval
        if labels.cat.categories[iname] in labels.cat.codes:
            mask = labels.cat.categories[iname] == labels.cat.codes
        else:
            mask = iname == labels.cat.codes
        groups_masks[iname] = mask.values
    groups_ids = list(range(len(groups_order)))
    if groups_order_subset != 'all':
        groups_ids = []
        for name in groups_order_subset:
            groups_ids.append(
                cp.where(cp.array(labels.cat.categories.to_array().astype("int32")) == int(name))[0][0]
            )
        if len(groups_ids) == 0:
            # fallback to index retrieval
            groups_ids = cp.where(
                cp.in1d(
                    cp.arange(len(labels.cat.categories)).astype(str),
                    cp.array(groups_order_subset),
                )
            )[0]
            
        groups_ids = [groups_id.item() for groups_id in groups_ids]
        groups_masks = groups_masks[groups_ids]
        groups_order_subset = labels.cat.categories[groups_ids].to_array()
    else:
        groups_order_subset = groups_order.to_array()
    return groups_order_subset, groups_masks



In [35]:
def _select_top_n(scores, n_top):
    n_from = scores.shape[0]
    reference_indices = np.arange(n_from, dtype=int)
    partition = np.argpartition(scores, -n_top)[-n_top:]
    partial_indices = np.argsort(scores[partition])[::-1]
    global_indices = reference_indices[partition][partial_indices]
    return global_indices

def _ranks(X, mask=None, mask_rest=None):
    CONST_MAX_SIZE = 10000000
    n_genes = X.shape[1]
    if issparse(X):
        merge = lambda tpl: vstack(tpl).toarray()
        adapt = lambda X: X.toarray()
    else:
        merge = np.vstack
        adapt = lambda X: X
    masked = mask is not None and mask_rest is not None
    if masked:
        n_cells = np.count_nonzero(mask) + np.count_nonzero(mask_rest)
        get_chunk = lambda X, left, right: merge(
            (X[mask, left:right], X[mask_rest, left:right])
        )
    else:
        n_cells = X.shape[0]
        get_chunk = lambda X, left, right: adapt(X[:, left:right])
    # Calculate chunk frames
    max_chunk = floor(CONST_MAX_SIZE / n_cells)
    for left in range(0, n_genes, max_chunk):
        right = min(left + max_chunk, n_genes)
        df = pd.DataFrame(data=get_chunk(X, left, right))
        ranks = df.rank()
        yield ranks, left, right

def sc_select_groups(adata, groups_order_subset='all', key='groups'):
    """Get subset of groups in adata.obs[key].
    """
    groups_order = adata.obs[key].cat.categories
    if key + '_masks' in adata.uns:
        groups_masks = adata.uns[key + '_masks']
    else:
        groups_masks = np.zeros(
            (len(adata.obs[key].cat.categories), adata.obs[key].values.size), dtype=bool
        )
        for iname, name in enumerate(adata.obs[key].cat.categories):
            # if the name is not found, fallback to index retrieval
            if adata.obs[key].cat.categories[iname] in adata.obs[key].values:
                mask = adata.obs[key].cat.categories[iname] == adata.obs[key].values
            else:
                mask = str(iname) == adata.obs[key].values
            groups_masks[iname] = mask
    groups_ids = list(range(len(groups_order)))
    if groups_order_subset != 'all':
        groups_ids = []
        for name in groups_order_subset:
            groups_ids.append(
                np.where(adata.obs[key].cat.categories.values == name)[0][0]
            )
        if len(groups_ids) == 0:
            # fallback to index retrieval
            groups_ids = np.where(
                np.in1d(
                    np.arange(len(adata.obs[key].cat.categories)).astype(str),
                    np.array(groups_order_subset),
                )
            )[0]
        if len(groups_ids) == 0:
            logg.debug(
                f'{np.array(groups_order_subset)} invalid! specify valid '
                f'groups_order (or indices) from {adata.obs[key].cat.categories}',
            )
            from sys import exit

            exit(0)
        groups_masks = groups_masks[groups_ids]
        groups_order_subset = adata.obs[key].cat.categories[groups_ids].values
    else:
        groups_order_subset = groups_order.values
    return groups_order_subset, groups_masks

## DGE when groups do not include 0

In [36]:
cluster_labels = cudf.Series.from_categorical(adata.obs["louvain"].cat)
var_names = cudf.Series(var_names)
scores, names, reference = rapids_scanpy_funcs.rank_genes_groups(
    sparse_gpu_array, cluster_labels, var_names, n_genes=1, groups=['1', '2', '3'], reference='4')
print(scores)
print(names)

Ranking took (GPU): 1.2022655010223389
Preparing output np.rec.fromarrays took (CPU): 0.0002472400665283203
Note: This operation will be accelerated in a future version
[(0.0411449, 0.05192262, 0.0491985, 0.04489372)]
[('HLA-DRB1', 'CLDN5', 'HLA-DRB1', 'IFI27')]


In [37]:
sc.tl.rank_genes_groups(adata, groupby="louvain", n_genes=1, groups=['1', '2', '3'], reference='4', method='logreg', use_raw=False)
adata.uns['rank_genes_groups']

{'params': {'groupby': 'louvain',
  'reference': '4',
  'method': 'logreg',
  'use_raw': False,
  'layer': None,
  'corr_method': 'benjamini-hochberg'},
 'scores': rec.array([(0.09784701, 0.06829986, 0.14802197, 0.0592204)],
           dtype=[('1', '<f4'), ('2', '<f4'), ('3', '<f4'), ('4', '<f4')]),
 'names': rec.array([('EDNRB', 'LOC731424', 'ACKR1', 'COX4I2')],
           dtype=[('1', '<U50'), ('2', '<U50'), ('3', '<U50'), ('4', '<U50')])}

In [38]:
groups=['1', '2', '3']
labels = cluster_labels
n_genes=1
reference='4'
groups_order = list(groups)
groups_order += [reference]
n_vars = len(var_names)
n_genes_user = n_genes
reference = groups_order[0]
sc_groups_order, sc_groups_masks = sc_select_groups(adata, groups_order, 'louvain')
sc_grouping_mask = adata.obs['louvain'].isin(sc_groups_order)
sc_grouping = adata.obs.loc[sc_grouping_mask, 'louvain']
sc_X = adata.X[sc_grouping_mask.values, :]
print(sc_X.shape)

(12933, 5000)


In [39]:
y = np.array(sc_grouping.cat.codes).astype('float32')
print(y)
print(np.unique(y))
print(y.shape)

[1. 1. 1. ... 4. 4. 4.]
[1. 2. 3. 4.]
(12933,)


#### With penalty

In [41]:
clf = sklearn.linear_model.LogisticRegression()
clf.fit(sc_X, y)
sc_scores_all = clf.coef_
print(sc_scores_all.shape)
print(sc_scores_all[0])
r_clf = cuml.linear_model.LogisticRegression(max_iter=100)
r_clf.fit(sc_X, y)
r_scores_all = cp.array(r_clf.coef_).T
print(r_scores_all.shape)
print(r_scores_all[0])
for i in np.unique(y).astype(int)[:-1]:
    print(np.corrcoef(r_scores_all[i].get(), sc_scores_all[i])[1,0])

(4, 5000)
[ 7.38160557e-05 -1.17872502e-02 -8.82218858e-03 ...  7.79808874e-04
 -4.77411688e-03 -2.60756054e-04]
(4, 5000)
[ 0.00102803  0.00186996 -0.06843644 ...  0.00165118 -0.00606385
  0.0012012 ]
0.04647692737498638
0.11506294857788285
-0.8902980580225195


#### Without penalty

In [40]:
clf = sklearn.linear_model.LogisticRegression(penalty="none")
clf.fit(sc_X, y)
sc_scores_all = clf.coef_
print(sc_scores_all.shape)
print(sc_scores_all[0])
r_clf = cuml.linear_model.LogisticRegression(penalty="none", max_iter=100)
r_clf.fit(sc_X, y)
r_scores_all = cp.array(r_clf.coef_).T
print(r_scores_all.shape)
print(r_scores_all[0])
for i in np.unique(y).astype(int)[:-1]:
    print(np.corrcoef(r_scores_all[i].get(), sc_scores_all[i])[1,0])

(4, 5000)
[ 7.76778326e-05 -2.75741480e-02 -1.30746533e-02 ...  1.21806396e-03
 -8.19390005e-03 -5.96352474e-04]
(4, 5000)
[1.0003258 1.0019699 0.9039301 ... 1.0020329 0.988751  1.000614 ]
-0.33368533377981796
-0.06748994934690329
-0.5755117354642383


## DGE when groups include 0 but are not continuous with reference

In [42]:
cluster_labels = cudf.Series.from_categorical(adata.obs["louvain"].cat)
var_names = cudf.Series(var_names)
scores, names, reference = rapids_scanpy_funcs.rank_genes_groups(
    sparse_gpu_array, cluster_labels, var_names, n_genes=1, groups=['0', '1', '2'], reference='4')
print(scores)
print(names)

Ranking took (GPU): 1.3929574489593506
Preparing output np.rec.fromarrays took (CPU): 0.00026917457580566406
Note: This operation will be accelerated in a future version
[(0.05223215, 0.04582195, 0.04089808, 0.02837277)]
[('FCN3', 'IL1RL1', 'HLA-DRB1', 'CD9')]


In [43]:
sc.tl.rank_genes_groups(adata, groupby="louvain", n_genes=1, groups=['0', '1', '2'], reference='4', method='logreg', use_raw=False)
adata.uns['rank_genes_groups']

/rapids-genomics/rapidgenomics/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


{'params': {'groupby': 'louvain',
  'reference': '4',
  'method': 'logreg',
  'use_raw': False,
  'layer': None,
  'corr_method': 'benjamini-hochberg'},
 'scores': rec.array([(0.14119661, 0.22543633, 0.07103329, 0.0609046)],
           dtype=[('0', '<f4'), ('1', '<f4'), ('2', '<f4'), ('4', '<f4')]),
 'names': rec.array([('FCN3', 'EDNRB', 'LOC731424', 'PDGFRB')],
           dtype=[('0', '<U50'), ('1', '<U50'), ('2', '<U50'), ('4', '<U50')])}

In [44]:
groups=['0', '1', '2']
labels = cluster_labels
n_genes=1
reference='4'
groups_order = list(groups)
groups_order += [reference]
n_vars = len(var_names)
n_genes_user = n_genes
reference = groups_order[0]
sc_groups_order, sc_groups_masks = sc_select_groups(adata, groups_order, 'louvain')
sc_grouping_mask = adata.obs['louvain'].isin(sc_groups_order)
sc_grouping = adata.obs.loc[sc_grouping_mask, 'louvain']
sc_X = adata.X[sc_grouping_mask.values, :]
print(sc_X.shape)

(17240, 5000)


In [45]:
y = np.array(sc_grouping.cat.codes).astype('float32')
print(y)
print(np.unique(y))
print(y.shape)

[1. 1. 1. ... 4. 4. 4.]
[0. 1. 2. 4.]
(17240,)


#### With penalty

In [47]:
clf = sklearn.linear_model.LogisticRegression()
clf.fit(sc_X, y)
sc_scores_all = clf.coef_
print(sc_scores_all.shape)
print(sc_scores_all[0])
r_clf = cuml.linear_model.LogisticRegression(max_iter=100)
r_clf.fit(sc_X, y)
r_scores_all = cp.array(r_clf.coef_).T
print(r_scores_all.shape)
print(r_scores_all[0])
for i in np.unique(y).astype(int)[:-1]:
    print(np.corrcoef(r_scores_all[i].get(), sc_scores_all[i])[1,0])

/rapids-genomics/rapidgenomics/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


(4, 5000)
[ 6.72163261e-06 -7.25809751e-04  8.63246486e-03 ...  8.83177811e-05
 -1.45885428e-03 -1.13245907e-05]
(4, 5000)
[-6.0822052e-04 -5.4322236e-04 -4.7463972e-02 ... -9.9496603e-05
 -5.7657333e-03 -4.7655631e-04]
0.3168691833084048
0.26329726674826176
0.2996652452141231


#### Without penalty

In [46]:
clf = sklearn.linear_model.LogisticRegression(penalty="none")
clf.fit(sc_X, y)
sc_scores_all = clf.coef_
print(sc_scores_all.shape)
print(sc_scores_all[0])
r_clf = cuml.linear_model.LogisticRegression(penalty="none", max_iter=100)
r_clf.fit(sc_X, y)
r_scores_all = cp.array(r_clf.coef_).T
print(r_scores_all.shape)
print(r_scores_all[0])
for i in np.unique(y).astype(int)[:-1]:
    print(np.corrcoef(r_scores_all[i].get(), sc_scores_all[i])[1,0])

(4, 5000)
[ 5.11931547e-06  1.74696262e-03  2.59674919e-02 ...  1.26189393e-04
 -8.06942837e-03  5.34965609e-05]
(4, 5000)
[1.000049   1.000179   0.9813838  ... 1.0005978  1.0008705  0.99990064]
0.5644554040397153
0.5077624088077448
0.6797220213991721


## O in groups and groups continuous with reference

In [48]:
groups=['0', '1', '2', '3']
labels = cluster_labels
n_genes=1
reference='4'
groups_order = list(groups)
groups_order += [reference]
n_vars = len(var_names)
n_genes_user = n_genes
reference = groups_order[0]
sc_groups_order, sc_groups_masks = sc_select_groups(adata, groups_order, 'louvain')
sc_grouping_mask = adata.obs['louvain'].isin(sc_groups_order)
sc_grouping = adata.obs.loc[sc_grouping_mask, 'louvain']
sc_X = adata.X[sc_grouping_mask.values, :]
print(sc_X.shape)

(18361, 5000)


In [49]:
y = np.array(sc_grouping.cat.codes).astype('float32')
print(y)
print(np.unique(y))
print(y.shape)

[1. 1. 1. ... 4. 4. 4.]
[0. 1. 2. 3. 4.]
(18361,)


#### With penalty

In [51]:
clf = sklearn.linear_model.LogisticRegression()
clf.fit(sc_X, y)
sc_scores_all = clf.coef_
print(sc_scores_all.shape)
print(sc_scores_all[0])
r_clf = cuml.linear_model.LogisticRegression(max_iter=100)
r_clf.fit(sc_X, y)
r_scores_all = cp.array(r_clf.coef_).T
print(r_scores_all.shape)
print(r_scores_all[0])
for i in np.unique(y).astype(int)[:-1]:
    print(np.corrcoef(r_scores_all[i].get(), sc_scores_all[i])[1,0])

/rapids-genomics/rapidgenomics/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


(5, 5000)
[ 6.18465405e-05 -5.46341211e-03  1.75755151e-02 ...  1.10172463e-03
 -9.49826100e-03 -3.25159297e-05]
(5, 5000)
[ 7.76682555e-06 -2.96154845e-04  2.80773849e-03 ...  1.05273386e-04
  6.32267212e-04 -2.14753618e-05]
0.7958349255273626
0.8252592670042664
0.948227769438734
0.7808189589684275


#### Without penalty

In [50]:
clf = sklearn.linear_model.LogisticRegression(penalty="none")
clf.fit(sc_X, y)
sc_scores_all = clf.coef_
print(sc_scores_all.shape)
print(sc_scores_all[0])
r_clf = cuml.linear_model.LogisticRegression(penalty="none", max_iter=100)
r_clf.fit(sc_X, y)
r_scores_all = cp.array(r_clf.coef_).T
print(r_scores_all.shape)
print(r_scores_all[0])
for i in np.unique(y).astype(int)[:-1]:
    print(np.corrcoef(r_scores_all[i].get(), sc_scores_all[i])[1,0])

(5, 5000)
[ 0.00017013 -0.00585557  0.05048242 ...  0.00246908 -0.02113546
  0.00016952]
(5, 5000)
[1.0000008 1.0013689 1.0112766 ... 1.0002273 1.0016681 0.9999429]
0.9255306004941825
0.9199888159984593
0.7951273178705532
0.9110552909582099


## Using 'all' vs. 'rest' in DGE

In [52]:
cluster_labels = cudf.Series.from_categorical(adata.obs["louvain"].cat)
var_names = cudf.Series(var_names)
scores, names, reference = rapids_scanpy_funcs.rank_genes_groups(
    sparse_gpu_array, cluster_labels, var_names, n_genes=1, groups='all', reference='rest')
print(scores)
print(names)

Ranking took (GPU): 5.249251127243042
Preparing output np.rec.fromarrays took (CPU): 0.001451730728149414
Note: This operation will be accelerated in a future version
[(0.0297264, 0.04210025, 0.04450222, 0.0431834, 0.02540051, 0.03570574, 0.04485831, 0.03297769, 0.03592882, 0.02816268, 0.02594503, 0.04435266, 0.03464667, 0.0390257, 0.0127864, 0.03279061, 0.03292505, 0.03827792, 0.02789916, 0.03810845, 0.04725059, 0.02496053, 0.04091164, 0.03817671, 0.02771649, 0.03709015, 0.00440436, 0.03318343, 0.02671634, 0.02320579, 0.02570026, 0.02966261, 0.01157081, 0.03337426, 0.02384854)]
[('AKAP12', 'IL1RL1', 'CCL20', 'ACKR1', 'PTN', 'SCGB3A2', 'MYOM2', 'GZMK', 'RETN', 'SLC6A14', 'MZB1', 'CD79A', 'CD2', 'TPSAB1', 'MSR1', 'CCL18', 'KIAA0101', 'CLEC10A', 'ZNF683', 'KLRC2', 'FCN1', 'PLVAP', 'DKK2', 'CCL21', 'RGS5', 'CCR7', 'PPBP', 'CHIAP2', 'SOSTDC1', 'FBLN1', 'ASPN', 'AGER', 'TMEM190', 'KRT17', 'ACTA2')]


In [53]:
sc.tl.rank_genes_groups(adata, groupby="louvain", n_genes=1, groups='all', reference='rest', method='logreg', use_raw=False)
adata.uns['rank_genes_groups']

/rapids-genomics/rapidgenomics/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


{'params': {'groupby': 'louvain',
  'reference': 'rest',
  'method': 'logreg',
  'use_raw': False,
  'layer': None,
  'corr_method': 'benjamini-hochberg'},
 'scores': rec.array([(0.22648402, 0.4015468, 0.4570785, 0.39126313, 0.14303766, 0.32053962, 0.49248633, 0.5683104, 0.46839613, 0.27672288, 0.26343212, 0.33692372, 0.28136167, 0.2660846, 0.35119945, 0.47345528, 0.34931272, 0.36503047, 0.29844698, 0.42254415, 0.26805136, 0.27768624, 0.36269936, 0.2483729, 0.25564334, 0.40181407, 0.06370448, 0.30314136, 0.17175293, 0.12544572, 0.1956848, 0.24868828, 0.08667456, 0.2519547, 0.14818831)],
           dtype=[('0', '<f4'), ('1', '<f4'), ('2', '<f4'), ('3', '<f4'), ('4', '<f4'), ('5', '<f4'), ('6', '<f4'), ('7', '<f4'), ('8', '<f4'), ('9', '<f4'), ('10', '<f4'), ('11', '<f4'), ('12', '<f4'), ('13', '<f4'), ('14', '<f4'), ('15', '<f4'), ('16', '<f4'), ('17', '<f4'), ('18', '<f4'), ('19', '<f4'), ('20', '<f4'), ('21', '<f4'), ('22', '<f4'), ('23', '<f4'), ('24', '<f4'), ('25', '<f4'), ('26', '

In [54]:
groups='all'
labels = cluster_labels
reference='rest'
groups_order = 'all'
n_vars = len(var_names)
n_genes_user = n_genes
sc_groups_order, sc_groups_masks = sc_select_groups(adata, groups_order, 'louvain')
sc_grouping_mask = adata.obs['louvain'].isin(sc_groups_order)
sc_grouping = adata.obs.loc[sc_grouping_mask, 'louvain']
sc_X = adata.X[sc_grouping_mask.values, :]
print(sc_X.shape)

(65462, 5000)


In [55]:
y = np.array(sc_grouping.cat.codes).astype('float32')
print(y)
print(np.unique(y))
print(y.shape)

[ 1.  1.  1. ...  4.  4. 29.]
[ 0.  1.  2.  3.  4.  5.  6.  7.  8.  9. 10. 11. 12. 13. 14. 15. 16. 17.
 18. 19. 20. 21. 22. 23. 24. 25. 26. 27. 28. 29. 30. 31. 32. 33. 34.]
(65462,)


#### Without penalty

In [56]:
clf = sklearn.linear_model.LogisticRegression(penalty="none")
clf.fit(sc_X, y)
sc_scores_all = clf.coef_
print(sc_scores_all.shape)
print(sc_scores_all[0])
r_clf = cuml.linear_model.LogisticRegression(penalty="none", max_iter=100)
r_clf.fit(sc_X, y)
r_scores_all = cp.array(r_clf.coef_).T
print(r_scores_all.shape)
print(r_scores_all[0])
for i in np.unique(y).astype(int)[:-1]:
    print(np.corrcoef(r_scores_all[i].get(), sc_scores_all[i])[1,0])

(35, 5000)
[ 0.00063807  0.18657404  0.08970132 ...  0.00376566 -0.05563251
 -0.00544399]
(35, 5000)
[1.0001405  1.0124428  1.0033175  ... 1.0008746  1.0149763  0.99902433]
0.9253282309330967
0.9544669010820406
0.7820028332301421
0.9455403749055326
0.9667657856522671
0.8612928034647898
0.903407911845778
0.8876006379246725
0.7287760721661954
0.8380529790080212
0.9386897277128413
0.9508854400311298
0.8557299841649896
0.9417650232022562
0.7369478878895662
0.7141608149849421
0.8475546410378201
0.8247734897125875
0.9376467556596224
0.8669446576176999
0.9037282988965852
0.9279164646313476
0.9514197526747686
0.9948367200326989
0.9289575710468306
0.8978306930140919
0.994869245223436
0.7541610207152724
0.964967662196213
0.956821235301046
0.9648362075524639
0.9193246509397599
0.9660152576753225
0.8864207372400081


#### With penalty

In [57]:
clf = sklearn.linear_model.LogisticRegression()
clf.fit(sc_X, y)
sc_scores_all = clf.coef_
print(sc_scores_all.shape)
print(sc_scores_all[0])
r_clf = cuml.linear_model.LogisticRegression(max_iter=100)
r_clf.fit(sc_X, y)
r_scores_all = cp.array(r_clf.coef_).T
print(r_scores_all.shape)
print(r_scores_all[0])
for i in np.unique(y).astype(int)[:-1]:
    print(np.corrcoef(r_scores_all[i].get(), sc_scores_all[i])[1,0])

/rapids-genomics/rapidgenomics/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


(35, 5000)
[ 2.43902716e-06  4.92071258e-02 -2.06252870e-03 ... -1.77370259e-04
 -2.97540250e-03 -7.03491793e-04]
(35, 5000)
[ 4.2938169e-05  1.0489807e-04 -2.0886110e-03 ...  3.6847877e-04
  2.4463162e-03 -1.4279367e-04]
0.6785875748243684
0.7745571124479169
0.6215108306912447
0.7266008111697091
0.8356478482569802
0.5971683504105683
0.6007280340233107
0.5788348088001126
0.46562316783537894
0.6247829254625665
0.9259819927140803
0.8686898232267607
0.5699063304653919
0.9020053163400611
0.4734949212597546
0.47426925937131864
0.726548190819075
0.6770083945125206
0.7372150476136767
0.5091078558208798
0.7845694842101222
0.6685542303371479
0.760709724349164
0.9440590161960846
0.6558533947278549
0.7074766000197749
0.9348412256127454
0.529996801354135
0.7850096807501702
0.8413848125876612
0.8785760669509548
0.7454151338087236
0.9682919752569986
0.652453287947517


In [55]:
r_top_100 = np.argsort(r_scores_all[0])[-100:]

In [56]:
sc_top_100 = np.argsort(sc_scores_all[0])[-100:]

In [58]:
np.intersect1d(r_top_100.get(), sc_top_100)

array([], dtype=int64)

In [206]:
partition = cp.argpartition(r_scores_all[0], -1)[-1:]
print(partition)
print(var_names[partition], r_scores_all[0][partition])
r_scores_all[0].max()

[3666]
3666    SEPP1
dtype: object [1.0138866]


array(1.0138866, dtype=float32)

In [207]:
partition = np.argpartition(sc_scores_all[0], -1)[-1:]
print(partition)
print(var_names[partition], sc_scores_all[0][partition])
sc_scores_all[0].max()

[4628]
4628    GPIHBP1
dtype: object [0.19351941]


0.19351940748586458